<a href="https://colab.research.google.com/github/fpseverino/progetto-ml-ai/blob/main/Severino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, HalvingGridSearchCV
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import copy

# Classe Task
Viene definita una classe `Task`, che viene inizializzata a partire da un `DataFrame` e ha come attributi `X`, `y` e le loro suddivisioni per il training e il testing.

La classe contiene anche dei metodi per testare il `Task` con uno specifico classificatore e per scalare l'`X` con uno specifico scaler.

In [33]:
class Task:
  def __init__(self, df):
    self.X = df.drop("Label", axis = 1)
    self.X = self.X.drop("Task", axis = 1)
    self.X = self.X.drop("Id", axis = 1)
    self.y = df["Label"]

    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size = 0.3, random_state = 0)

  def test(self, clf):
    clf.fit(self.X_train, self.y_train)
    return clf.score(self.X_test, self.y_test)

  def scale(self, sc):
    sc.fit(self.X_train)
    self.X_train = sc.transform(self.X_train)
    self.X_test = sc.transform(self.X_test)

Di seguito vengono letti i `csv` dei Task già ripuliti e vengono salvati come oggetti `Task` all'interno dell'array `tasks`.

In [34]:
tasks = []
for i in range(11, 22):
  df = pd.read_csv("Task_{}_clean.csv".format(i))
  task = Task(df)
  tasks.append(task)

# Classificatore `AdaBoost`
Un classificatore AdaBoost è un meta-stimatore che inizia inserendo un classificatore nel set di dati originale e quindi adatta copie aggiuntive del classificatore nello stesso set di dati, ma dove i pesi delle istanze classificate in modo errato vengono regolati in modo tale che i classificatori successivi si concentrino maggiormente sui casi difficili.

Viene eseguito un primo test sui task normali e successivamente un altro test sui task scalati per mezzo di un `MinMaxScaler`.

In [21]:
clf = AdaBoostClassifier(n_estimators = 100, random_state = 0)

scores = []
for task in tasks:
  scores.append(task.test(clf))
print("score:", np.mean(scores))

sc = MinMaxScaler(feature_range = (0, 1))
scaled_tasks = copy.deepcopy(tasks)
for task in scaled_tasks:
  task.scale(sc)

scaled_scores = []
for task in scaled_tasks:
  scaled_scores.append(task.test(clf))
print("Scaled score:", np.mean(scaled_scores))

score: 0.7904040404040404
Scaled score: 0.8257575757575758


# Classificatore Gradient Boosting
Questo algoritmo costruisce un modello additivo in modo graduale; consente l'ottimizzazione di funzioni di perdita differenziabili arbitrarie. In ogni fase, `n_classes_` alberi di regressione sono adattati al gradiente negativo della funzione di perdita, ad es. perdita di log binaria o multiclasse.

Viene eseguito un primo test sui task normali e successivamente un altro test sui task scalati per mezzo di un `MinMaxScaler`.

In [12]:
clf = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.01, random_state = 0)

scores = []
for task in tasks:
  scores.append(task.test(clf))
print("score:", np.mean(scores))

sc = MinMaxScaler(feature_range = (0, 1))
scaled_tasks = copy.deepcopy(tasks)
for task in scaled_tasks:
  task.scale(sc)

scaled_scores = []
for task in scaled_tasks:
  scaled_scores.append(task.test(clf))
print("Scaled score:", np.mean(scaled_scores))

score: 0.8762626262626263
Scaled score: 0.8712121212121212


# Albero di classificazione Gradient Boosting basato sugli istogrammi
Questo stimatore ha il supporto nativo per i valori mancanti (`NaN`). Durante l'addestramento, il coltivatore di alberi impara ad ogni punto di divisione se i campioni con valori mancanti devono andare al figlio sinistro o destro, in base al potenziale guadagno. Durante la previsione, i campioni con valori mancanti vengono assegnati di conseguenza al figlio sinistro o destro. Se durante l'addestramento non sono stati rilevati valori mancanti per una determinata funzionalità, i campioni con valori mancanti vengono mappati a qualunque figlio abbia il maggior numero di campioni.

Viene eseguito un primo test sui task normali e successivamente un altro test sui task scalati per mezzo di un `MinMaxScaler`.

In [13]:
clf = HistGradientBoostingClassifier(random_state = 0)

scores = []
for task in tasks:
  scores.append(task.test(clf))
print("score:", np.mean(scores))

sc = MinMaxScaler(feature_range = (0, 1))
scaled_tasks = copy.deepcopy(tasks)
for task in scaled_tasks:
  task.scale(sc)

scaled_scores = []
for task in scaled_tasks:
  scaled_scores.append(task.test(clf))
print("Scaled score:", np.mean(scaled_scores))

score: 0.8661616161616162
Scaled score: 0.8661616161616162


# Classificatore Percettrone multi-strato
Vengono scalati i task tramite uno `StandardScaler`, in seguito viene eseguita una ricerca sui parametri forniti, e infine viene testato su tutti i task un `MLPCLassifier` con i parametri migliori.

In [36]:
sc = StandardScaler()
scaled_tasks = copy.deepcopy(tasks)
for task in scaled_tasks:
  task.scale(sc)

parametri_mlp = {
  'hidden_layer_sizes': [(150, 100, 50), (120, 80, 40), (100, 66, 33)],
  'solver': ['lbfgs', 'sgd', 'adam'],
  'alpha': [0.0001, 0.1, 100],
  'max_iter': [100, 200, 300]
}
grid_mlp = HalvingGridSearchCV(MLPClassifier(), parametri_mlp, n_jobs = -1, cv = 3)

scores = []
for task in scaled_tasks:
  grid_mlp.fit(task.X_train, task.y_train)

  clf = MLPClassifier(
    hidden_layer_sizes = grid_mlp.best_params_["hidden_layer_sizes"],
    solver = grid_mlp.best_params_["solver"],
    alpha = grid_mlp.best_params_["alpha"],
    max_iter = grid_mlp.best_params_["max_iter"],
    random_state = 0
  )

  scores.append(task.test(clf))
print("score:", np.mean(scores))

score: 0.7979797979797979
